# Alzheimer's Classification on ADNI1_Preprocessed Dataset
This notebook trains LeNet and AlexNet models on the preprocessed MRI data located under `adni-processed/ADNI1_Processed/`.

In [ ]:

import os
import numpy as np
import pandas as pd
import nibabel as nib
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt


## Load Diagnosis Metadata

In [ ]:

diagnosis_df = pd.read_csv('/kaggle/input/csv-file/ADNI1_Complete_1Yr_1.5T_6_20_2025.csv')
diagnosis_df = diagnosis_df[['Subject', 'Group']].drop_duplicates()
diagnosis_df = diagnosis_df[diagnosis_df['Group'].isin(['CN', 'MCI', 'AD'])]
diagnosis_map = dict(zip(diagnosis_df['Subject'], diagnosis_df['Group']))
label_map = {'CN': 0, 'MCI': 1, 'AD': 2}


## Load and Preprocess Middle Slices from Preprocessed NIfTI Files

In [ ]:

def load_middle_slice(path):
    img = nib.load(path).get_fdata()
    mid = img.shape[2] // 2
    slice_img = img[:, :, mid]
    slice_img = cv2.resize(slice_img, (64, 64))
    slice_img = (slice_img - np.mean(slice_img)) / (np.std(slice_img) + 1e-8)
    return np.expand_dims(slice_img, axis=-1)

X = []
y = []

data_dir = '/kaggle/input/adni-processed/ADNI1_Processed'
for subject_id in os.listdir(data_dir):
    subj_path = os.path.join(data_dir, subject_id)
    if subject_id not in diagnosis_map:
        continue
    label = label_map[diagnosis_map[subject_id]]
    for subfolder, _, files in os.walk(subj_path):
        for file in files:
            if file.endswith('.nii') or file.endswith('.nii.gz'):
                try:
                    full_path = os.path.join(subfolder, file)
                    img = load_middle_slice(full_path)
                    X.append(img)
                    y.append(label)
                    break  # one scan per subject
                except:
                    continue

X = np.array(X)
y = to_categorical(np.array(y), num_classes=3)
print(f"Loaded {X.shape[0]} samples with shape {X.shape[1:]}")


## Train-Test Split

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


## LeNet Model

In [ ]:

lenet = Sequential([
    Conv2D(6, kernel_size=5, activation='relu', input_shape=(64, 64, 1), padding='same'),
    AveragePooling2D(pool_size=(2, 2)),
    Conv2D(16, kernel_size=5, activation='relu'),
    AveragePooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(3, activation='softmax')
])
lenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lenet_history = lenet.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=16)


## AlexNet Model

In [ ]:

alexnet = Sequential([
    Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=(64, 64, 1), padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=3, strides=2),
    Conv2D(256, (5, 5), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=3, strides=2),
    Conv2D(384, (3, 3), activation='relu', padding='same'),
    Conv2D(384, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=3, strides=2),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])
alexnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
alexnet_history = alexnet.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=16)
